In [1]:
import pandas as pd
import tensorflow as tf
import glob
import os
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

import seaborn as sns
import helpers
import dataset
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-10-28 01:42:04.983534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = '/Users/frank/Downloads/dataSets/Trial-dataset/Clips/processed_lie/' #60 entries
truth_trial_path = '/Users/frank/Downloads/dataSets/Trial-dataset/Clips/processed_truth/' #61 entries

MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [4]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [11]:
LSTM_NEURONS = 125
DROPOUT = 0.2
EPOCHS = 40
OPTIM = 'adam'
LOSS = 'binary_crossentropy' # cross entropy/ we are doing regression, not classification!

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))
  model.add(Dropout(drop))
  model.add(Dense(1, activation='relu')) #sigmoid
  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)

MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)
MODEL.summary()

Epoch 1/40
1536/1536 [==============================] - 11s 6ms/step - loss: 0.6997 - accuracy: 0.5409 - val_loss: 0.6880 - val_accuracy: 0.5592
Epoch 2/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6898 - accuracy: 0.5519 - val_loss: 0.6870 - val_accuracy: 0.5591
Epoch 3/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6881 - accuracy: 0.5548 - val_loss: 0.6870 - val_accuracy: 0.5589
Epoch 4/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6871 - accuracy: 0.5602 - val_loss: 0.6923 - val_accuracy: 0.5204
Epoch 5/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6872 - accuracy: 0.5594 - val_loss: 0.6885 - val_accuracy: 0.5579
Epoch 6/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6869 - accuracy: 0.5606 - val_loss: 0.6867 - val_accuracy: 0.5591
Epoch 7/40
1536/1536 [==============================] - 9s 6ms/step - loss: 0.6865 - accuracy: 0.5632 - val_loss: 0.6863 - val_accuracy

KeyboardInterrupt: 